In [1]:
import pandas as pd
import numpy as np
import os
import glob
from power_to_temperature import *
from test import *

In [2]:
freqs,vals,bwidth = bandwidth(50.,90.)
Freqs = np.linspace(1e-22,250,32769)
eta_nu = eta(Freqs)
mask = (Freqs>=50)&(Freqs<=91)
bwidth

7629.161707711557

In [3]:
folders = glob.glob('Data/*')
folders.sort()
i=-1
for subdirs, dirs, files in os.walk('Data'):
    dirs.sort()
    files.sort()
    short,antenna,measure = [],[],[]
    short_date,antenna_date,measure_date=[],[],[]
    for file in files:
        path = os.path.join(subdirs,file)
        date = file.split("_")[0]
        if  os.path.getsize(path)==0:
            print 'EMPTY FILE:',path
            continue
        elif file.endswith('short.dat'):
            dB_short = np.loadtxt(path,unpack=True)
            T_short = Res2Temp(dB_short,bwidth)
            short.append(T_short),short_date.append(date)
        elif file.endswith('noise.dat'):
            dB_noise = np.loadtxt(path,unpack=True)
        elif file.endswith('antenna.dat'):
            dB_antenna = np.loadtxt(path,unpack=True)
            dB_clean = dB_antenna - dB_noise
            T_antenna = Radio_source_trans(dB_clean, Freqs, bwidth)
            T_measure = T_antenna/eta_nu - T_short
            antenna.append(T_antenna),measure.append(T_measure)
            antenna_date.append(date),measure_date.append(date)
            
    if i>=0 and i<len(folders) and antenna and short and measure:
        name = os.path.normpath(folders[i])
        name = name.split("/")[1]
        short = np.transpose(short)
        antenna = np.transpose(antenna)
        measure = np.transpose(measure)
        short_table = pd.DataFrame(short[mask], index = Freqs[mask], columns = short_date)
        short_table.to_hdf('Data_filter/short/'+name+'.hdf5','df')
        antenna_table = pd.DataFrame(antenna[mask], index = Freqs[mask], columns = antenna_date)
        antenna_table.to_hdf('Data_filter/antenna/'+name+'.hdf5','df')
        measure_table = pd.DataFrame(measure[mask], index = Freqs[mask], columns = measure_date)
        measure_table.to_hdf('Data_filter/Tmeas/'+name+'.hdf5','df')
    i+=1
    
    

EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-02-29_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-02-39_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-02-50_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-15-33_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-15-44_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-15-55_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-29-28_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-29-39_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-29-50_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-45-18_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-45-29_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-45-40_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-03/2013-06-14-03-04-54_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-03/2013-06-14-03-05-05_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-03/2013-06-14-03-05-

IndexError: list index out of range

In [4]:
day14 = glob.glob('Data_filter/Tmeas/2013-06-14*')
dfs = []
for path in day14:
    dfs.append(pd.read_hdf(path))
big_frame1 = pd.concat(dfs, axis = 1)
big_frame1.to_hdf('calibration/Day14.hdf5','df')

In [5]:
day15 = glob.glob('Data_filter/Tmeas/2013-06-15*')
dfs = []
for path in day15:
    dfs.append(pd.read_hdf(path))
big_frame2 = pd.concat(dfs, axis = 1)
big_frame2.to_hdf('calibration/Day15.hdf5','df')

In [6]:
all_data = [big_frame1,big_frame2]
day14_15 = pd.concat(all_data,axis=1)
day14_15.to_hdf('calibration/Days14_15.hdf5','df')